In [62]:
from platform import python_version
import pandas as pd
import numpy as np 
import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.multioutput import MultiOutputRegressor
import lightgbm as lgb
import xgboost as xgb

print(f'python {python_version()}')
print(f'pandas {pd.__version__}')
print(f'numpy {np.__version__}')

python 3.6.10
pandas 1.0.5
numpy 1.19.0


In [8]:
data = pd.read_csv('../input/MoltenPool_Datasheet_v1.1_csv.csv')

In [9]:
data.head(5)

,Power(W),Speed,Max1,Max2,Max3,Max4,Max5,Min1,Min2,Min3,Min4,Min5,Max Avg,Min Avg,Avg,Gap,Stabilty,Spattering,Keyhole induced pore
0,110,2200,30.01,30.00,30.08,29.42,30.77,17.15,17.86,16.43,15.37,16.79,30.06,16.72,23.39,13.34,0.2,1.0,1.0
1,140,2200,40.81,41.93,42.88,41.43,42.90,30.48,31.91,31.43,30.72,30.00,41.99,30.91,36.45,11.08,0.3,0.0,1.0
2,180,2200,46.80,47.68,48.21,46.97,47.15,32.98,30.71,33.58,30.71,30.92,47.36,31.78,39.57,15.58,0.3,0.5,1.0
3,210,2200,49.33,50.37,51.08,49.67,50.73,44.59,44.08,44.56,44.84,44.79,50.24,44.57,47.40,5.66,0.4,1.0,1.0
4,250,2200,56.43,56.32,56.10,56.80,56.11,48.23,46.80,48.21,47.52,47.87,56.35,47.73,52.04,8.63,0.3,0.5,1.0


In [12]:
indexName = data.columns.tolist()
indexName

['Power(W)',
 'Speed',
 'Max1',
 'Max2',
 'Max3',
 'Max4',
 'Max5',
 'Min1',
 'Min2',
 'Min3',
 'Min4',
 'Min5',
 'Max Avg',
 'Min Avg',
 'Avg',
 'Gap',
 'Stabilty',
 'Spattering',
 'Keyhole induced pore']

In [15]:
eraseName = ['Max1','Max2','Max3','Max4','Max5','Min1','Min2','Min3','Min4','Min5']

In [27]:
data = data.drop(eraseName,axis=1)

## Data Analysis 

In [35]:
copyData = data.copy()
x_data_list = ['Power(W)','Speed']
dummy_erase_list = ['Keyhole induced pore']

In [45]:
x_data = copyData[x_data_list]
y_data = copyData.drop(erase_list, axis=1)

ValueError: Need to specify at least one of 'labels', 'index' or 'columns'

In [46]:
y_data

,Max Avg,Min Avg,Avg,Gap,Stabilty,Spattering,Keyhole induced pore
0,30.06,16.72,23.39,13.34,0.2,1.0,1.0
1,41.99,30.91,36.45,11.08,0.3,0.0,1.0
2,47.36,31.78,39.57,15.58,0.3,0.5,1.0
3,50.24,44.57,47.40,5.66,0.4,1.0,1.0
4,56.35,47.73,52.04,8.63,0.3,0.5,1.0
...,...,...,...,...,...,...,...
59,82.86,77.24,80.05,5.62,0.8,0.5,0.1
60,96.52,85.59,91.06,10.94,0.9,0.5,0.4
61,138.72,120.87,129.80,17.85,0.9,0.5,0.2
62,148.40,127.52,137.96,20.87,1.0,0.5,0.3


In [56]:
def custom_metric(y_true, y_pred,**kwargs):
    return np.abs(np.array(y_true)-np.array(y_pred)).mean()

In [57]:
my_scorer = make_scorer(custom_metric, greater_is_better=True, n_jobs=-1)

# MultiOutputRegressor - lightgbm 

In [58]:
from sklearn.multioutput import MultiOutputRegressor
import lightgbm as lgb

In [66]:
model = MultiOutputRegressor(lgb.LGBMRegressor(random_state=42), n_jobs=-1)
model.fit(x_data, y_data)
#scores_lgbm = cross_val_score(model, x_data, y_data, cv=10,  verbose=2,scoring=my_scorer)
scores_lgbm = cross_val_score(model, x_data, y_data, cv=10,  verbose=2,scoring='r2')

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  ................................................................
[CV] ................................................. , total=   2.3s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.2s remaining:    0.0s


[CV] ................................................. , total=   0.5s
[CV]  ................................................................
[CV] ................................................. , total=   0.5s
[CV]  ................................................................
[CV] ................................................. , total=   0.5s
[CV]  ................................................................
[CV] ................................................. , total=   0.5s
[CV]  ................................................................
[CV] ................................................. , total=   0.7s
[CV]  ................................................................
[CV] ................................................. , total=   0.7s
[CV]  ................................................................
[CV] ................................................. , total=   0.6s
[CV]  ................................................................
[CV] .

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    7.5s finished


In [68]:
print("전체 score = {}".format(scores_lgbm))
print("평균 낸 score = {}".format(scores_lgbm.mean()))

전체 score = [-5.02704063 -5.53665526  0.28475937  0.19060069  0.1386485   0.23815307
 -1.73136863  0.54459456 -0.65541093 -1.00755204]
평균 낸 score = -1.256127129017724


# MultiOutputRegressor - xgboost

In [69]:
model = MultiOutputRegressor(xgb.XGBRegressor(random_state=42))
model.fit(x_data, y_data)
#scores_xgb = cross_val_score(model, x_data, y_data, cv=10,  verbose=2, scoring=my_scorer)
scores_xgb = cross_val_score(model, x_data, y_data, cv=10,  verbose=2,  scoring='r2')

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[CV]  ................................................................
[CV] ................................................. , total=   0.2s
[CV]  ................................................................
[CV] ................................................. , total=   1.0s
[CV]  ................................................................
[CV] ................................................. , total=   2.4s
[CV]  ................................................................
[CV] ................................................. , total=   1.6s
[CV]  ................................................................
[CV] ................................................. , total=   0.9s
[CV]  ................................................................
[CV] ................................................. , total=   0.1s
[CV]  ................................................................
[CV] ................................................. , total=   0.2s
[CV]  

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   13.8s finished


In [70]:
print("전체 score = {}".format(scores_xgb))
print("평균 낸 score = {}".format(scores_xgb.mean()))

전체 score = [-0.52930133 -0.74492556  0.4337263   0.61904087 -0.15943214 -0.61197888
 -1.82875859  0.39084774 -2.13178148 -0.02163567]
평균 낸 score = -0.4584198744480501
